In [148]:
import pandas as pd
import nltk
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


### Opening the nursery rhymes and separating them based on line

In [1]:
file = open('nursery_rhymes.txt','r')
with open('nursery_rhymes.txt') as f:
    contents = f.read()
    data=contents.split('\n')

In [150]:
len(data)

5071

In [151]:
data = " ".join(data)

### Cleaning data and separating by individual words

In [152]:
def clean_text(doc):
    tokens = doc.split()
    #Removing punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    #Removes non alphabet characters
    tokens = [word for word in tokens if word.isalpha()]
    #Makes everything lowercase
    tokens = [word.lower() for word in tokens]
    return tokens

In [153]:
tokens = clean_text(data)
print(tokens[:10])

['the', 'queen', 'of', 'hearts', 'the', 'queen', 'of', 'hearts', 'she', 'made']


I can now see that the data has been tokenized and now it is easy to read with no random characters included with everything in a unliform character set

In [154]:
print("There are: "+ str(len(set(tokens)))+ " unique words")
print("There are: "+ str(len(tokens))+ " total words")

There are: 2444 unique words
There are: 16786 total words


In [155]:
length = 10 + 1
lines = []
#Appending 10 words to a line that is fed in to create a sequence where the model will then determine what the following word 
# will be until it has finished the nursery Rhyme
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i>200000:
        break
print(len(lines))

16775


In [156]:
lines[3]

'hearts the queen of hearts she made some tarts all on'

### Assigning integer values to each word so it can be fed into the model

In [157]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [158]:
sequences=np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [159]:
wordDictSize = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=wordDictSize)
sequence_length = X.shape[1]

### Building model 

In [160]:
print(sequence_length)
model = Sequential()
model.add(Embedding(wordDictSize, 50, input_length = sequence_length))
model.add(LSTM(100, return_sequences= True))
model.add(LSTM(100))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(wordDictSize, activation='softmax'))
model.summary()

10
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 10, 50)            122250    
                                                                 
 lstm_18 (LSTM)              (None, 10, 100)           60400     
                                                                 
 lstm_19 (LSTM)              (None, 100)               80400     
                                                                 
 dense_18 (Dense)            (None, 100)               10100     
                                                                 
 dense_19 (Dense)            (None, 2445)              246945    
                                                                 
Total params: 520,095
Trainable params: 520,095
Non-trainable params: 0
_________________________________________________________________


### Compile and fit

In [161]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [163]:
model.fit(X, y, batch_size=256, epochs = 300)

Epoch 1/300
66/66 [==============================] - 4s 56ms/step - loss: 6.3838 - accuracy: 0.0547
Epoch 2/300
66/66 [==============================] - 4s 55ms/step - loss: 6.3002 - accuracy: 0.0547
Epoch 3/300
66/66 [==============================] - 4s 55ms/step - loss: 6.2838 - accuracy: 0.0532
Epoch 4/300
66/66 [==============================] - 4s 56ms/step - loss: 6.2538 - accuracy: 0.0545
Epoch 5/300
66/66 [==============================] - 4s 57ms/step - loss: 6.2311 - accuracy: 0.0549
Epoch 6/300
66/66 [==============================] - 4s 57ms/step - loss: 6.1898 - accuracy: 0.0548
Epoch 7/300
66/66 [==============================] - 4s 59ms/step - loss: 6.1012 - accuracy: 0.0549
Epoch 8/300
66/66 [==============================] - 4s 56ms/step - loss: 6.0307 - accuracy: 0.0550
Epoch 9/300
66/66 [==============================] - 4s 57ms/step - loss: 5.9534 - accuracy: 0.0576
Epoch 10/300
66/66 [==============================] - 4s 56ms/step - loss: 5.8493 - accuracy: 0.0636

66/66 [==============================] - 4s 57ms/step - loss: 0.3087 - accuracy: 0.9376
Epoch 163/300
66/66 [==============================] - 4s 57ms/step - loss: 0.3153 - accuracy: 0.9344
Epoch 164/300
66/66 [==============================] - 4s 57ms/step - loss: 0.3276 - accuracy: 0.9306
Epoch 165/300
66/66 [==============================] - 4s 57ms/step - loss: 0.3178 - accuracy: 0.9319
Epoch 166/300
66/66 [==============================] - 4s 57ms/step - loss: 0.3429 - accuracy: 0.9239
Epoch 167/300
66/66 [==============================] - 4s 58ms/step - loss: 0.3672 - accuracy: 0.9160
Epoch 168/300
66/66 [==============================] - 4s 57ms/step - loss: 0.3995 - accuracy: 0.9027
Epoch 169/300
66/66 [==============================] - 4s 58ms/step - loss: 0.3755 - accuracy: 0.9117
Epoch 170/300
66/66 [==============================] - 4s 58ms/step - loss: 0.3746 - accuracy: 0.9096
Epoch 171/300
66/66 [==============================] - 4s 59ms/step - loss: 0.3274 - accuracy: 0

I am actually really surprised with this accuracy as it is extremely high. 

### Creating the rhyme by generating text based on what the model learned

In [194]:
def create_rhyme(model, tokenizer, text_seq_length, seed, num_words):
    rhyme=[]
    for _ in range(num_words):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded=pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
        
        predict_y=model.predict(encoded) 
        y_pred=np.argmax(predict_y,axis=1)
        pred_word =''
        for word, index in tokenizer.word_index.items():
            if (index == y_pred):
                pred_word = word
                break
        seed = seed + " " + pred_word
        rhyme.append(pred_word)
    return ' '.join(rhyme)

This creates a rhyme by trying to predict the next word continuously until the number of words it generates is complete

In [195]:
# choosing random seed to make the nursery rhymes for 
import random
seedLen=(len(lines))
seedIndex=random.randrange(seedLen)
seed=lines[seedIndex]
#Creating Rhyme
raw_rhyme=create_rhyme(model, tokenizer, sequence_length, seed, 600)

### Formatting the rhyme to 30 lines with 20 words each

In [196]:
rhyme=raw_rhyme.split()
counter=0
temp=""
print("Model Generated Nursery Rhyme: ")
print(" ")
for i in range(len(rhyme)):
    temp+=rhyme[i]+" "
    if (i+1)%20==0 and i!=0:
        counter+=1
        print(temp)
        print("")
        temp=""

Model Generated Nursery Rhyme: 
 
woman went fortunetelling by cherrystones one i love two i love three i love i say four i love with 

all my heart five i cast away six he loves seven she loves eight both love nine he comes ten 

he tarries eleven he courts and twelve he marries little bopeep little bopeep has lost her sheep and cant tell 

where to find them let them alone and theyll come home and bring their tails behind them little bopeep fell 

fast asleep and dreamt she heard them bleating and when she awoke she found it a joke for still they 

were all fleeting then up she took her little crook determined for to find them she found them indeed but 

it made her heart bleed for theyd left all their tails behind them it happened one day as bopeep did 

stray into a meadow hard by there she espied their tails side by side all hung on a tree to 

dry she heaved a sigh and wiped her eye and went over hill and dale oh and tried what she 

could as a shepherdess should to tack to each 

While some of the rhymes dont make any sense, I was actually surprised that there was some rhymeing scheme in parts of it considering this was not written by a human